### Preprocessing with Cutoff Date

In [1]:
import pandas as pd

In [2]:
gstore = pd.read_csv("/Users/aoluwolerotimi/Datasets/train_dejsonified.csv")

/var/folders/z1/1zds15w97_g7_fhs4_83dc3m0000gn/T/ipykernel_43884/3136096564.py:1: DtypeWarning: Columns (2,54) have mixed types. Specify dtype option on import or set low_memory=False.
  gstore = pd.read_csv("/Users/aoluwolerotimi/Datasets/train_dejsonified.csv")


In [3]:
gstore.dtypes

channelGrouping                         object
date                                     int64
fullVisitorId                           object
sessionId                               object
socialEngagementType                    object
visitId                                  int64
visitNumber                              int64
visitStartTime                           int64
continent                               object
subContinent                            object
country                                 object
region                                  object
metro                                   object
city                                    object
cityId                                  object
networkDomain                           object
latitude                                object
longitude                               object
networkLocation                         object
browser                                 object
browserVersion                          object
browserSize  

In [4]:
gstore['date'] = pd.to_datetime(gstore['date'], format='%Y%m%d').dt.strftime('%Y-%m-%d')

In [5]:
# 6-month split is end of Feb 2017
print("Earliest Date:", (gstore['date']).min())
print("Latest Date:", (gstore['date']).max())

Earliest Date: 2016-08-01
Latest Date: 2017-08-01


In [ ]:
# beginning of august to end of Feb 
# beginning of march to beginning of august

In [6]:
cutoff_date = pd.to_datetime('2017-02-28')

In [10]:

gstore['date'] = pd.to_datetime(gstore['date'])

In [11]:
# Verify the updated data type
print(gstore['date'].dtype)

datetime64[ns]


In [12]:
df1 = gstore.loc[gstore['date'] <= cutoff_date]
df2 = gstore.loc[gstore['date'] > cutoff_date]


In [13]:
print(df1.shape[0] + df2.shape[0])

903653


In [14]:
# data collection cutoff dates
df1cutoff = pd.to_datetime('2016-11-30')
df2cutoff = pd.to_datetime('2017-06-30')

In [15]:
# set aside collection period and performance period data
df1p = df1.loc[df1['date'] > df1cutoff]

In [18]:
df1c = df1.loc[df1['date'] <= df1cutoff]

In [20]:
df1trainids = df1c['fullVisitorId'].unique().tolist()


In [16]:
import gc

In [21]:
del df1c
gc.collect()

31

In [23]:
df1c = pd.DataFrame(df1trainids, columns=['fullVisitorId'])